# Neural Nets from Scratch in Julia

## Lesson 12: Adding *Tensor* functionality

* In this video we'll add some additional functionality to the *Tensor* composite type.
* [Documentation site here](https://mikesaint-antoine.github.io/SimpleGrad.jl)
* [Github repo here](https://github.com/mikesaint-antoine/SimpleGrad.jl)

In [1]:
## code so far


struct Operation{FuncType, ArgTypes}
    op::FuncType
    args::ArgTypes
end


####################################################################################
###### Values

mutable struct Value{opType} <: Number
    data::Float64
    grad::Float64
    op::opType
end

# constructor -- Value(data, grad, op)
Value(x::Number) = Value(Float64(x), 0.0, nothing);


import Base.show
function show(io::IO, value::Value)
    print(io, "Value(",value.data,")")
end


import Base.==
function ==(a::Value, b::Value)
     return a===b
end


import Base.+
function +(a::Value, b::Value)

    out = a.data + b.data    
    result = Value(out, 0.0, Operation(+, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



backprop!(val::Value{Nothing}) = nothing


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(+), ArgTypes}
    
    # val = a + b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.grad
    val.op.args[2].grad += val.grad
    
end




function backward(a::Value)
    
    
    function build_topo(v::Value, visited=Value[], topo=Value[])
    
        if !(v in visited)
            
            push!(visited, v)
            
            if v.op != nothing
                for operand in v.op.args
                    
                    if operand isa Value
                        build_topo(operand, visited, topo)
                    end
                end 
            end
            
            push!(topo, v) 
            
            
        end
        return topo
    end
    
    
    
    topo = build_topo(a)
    
    a.grad = 1
    #da/da = 1
    
    for node in reverse(topo)
        backprop!(node)
    end
    
    
end


Base.promote_rule(::Type{<:Value}, ::Type{T}) where {T<:Number} = Value




import Base.*
function *(a::Value, b::Value)

    out = a.data * b.data    
    result = Value(out, 0.0, Operation(*, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(*), ArgTypes}
    
    # val = a * b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.op.args[2].data * val.grad    
    val.op.args[2].grad += val.op.args[1].data * val.grad
    
end



import Base.-

# negation
function -(a::Value)
    
    return a * -1
    
end

# subtraction
function -(a::Value, b::Value)
    
    return a + (-b)
    
    
end


import Base.inv
function inv(a::Value)
    
    out = 1.0 / a.data
    result = Value(out, 0.0, Operation(inv, (a,))) # Value(data, grad, op)
    return result # this should be a Value    
    
    
end


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(inv), ArgTypes}
    
    # val = inv(a)
    # update a.grad
    
    # a.grad -= (1.0 / a.data^2) * val.grad
    
    val.op.args[1].grad -= (1.0 / val.op.args[1].data^2) * val.grad
    
    
end


import Base./
function /(a::Value, b::Value)
     
    # a/b = a * b^(-1)
    
    return a * inv(b)
    
    
end


import Base.tanh
function tanh(a::Value)
    
    out = (exp(2 * a.data) - 1) / (exp(2 * a.data) + 1)
    result = Value(out, 0.0, Operation(tanh, (a,))) # Value(data, grad, op)
    return result # this should be a Value  
    
end




function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(tanh), ArgTypes}

    # val = tanh(a)
    # update a.grad
    
    val.op.args[1].grad += (1 - val.data^2) * val.grad
    

end

####################################################################################
###### Tensors

mutable struct Tensor{opType} <: AbstractArray{Float64, 2}
    data::Array{Float64,2}
    grad::Array{Float64,2}
    op::opType
end

# 2D constructor -- Tensor(data, grad, op)
Tensor(x::Array{Float64,2}) = Tensor(x, zeros(Float64,size(x)), nothing);

# 1D constructor
function Tensor(x::Array{Float64, 1}; column_vector::Bool=false)

    if column_vector
        # reshape x to column vector - size (N,1)
        data_2D = reshape(x, (length(x),1))

    else
        # DEFAULT - row vector - size (1,N)
        data_2D = reshape(x, (1, length(x)))

    end

    return Tensor(data_2D, zeros(Float64, size(data_2D)), nothing) # Tensor(data, grad, op)

end

Tensor

In [6]:
import Base.show
function show(io::IO, tensor::Tensor)
    print(io, "Tensor(",tensor.data,")")
end

backprop!(tensor::Tensor{Nothing}) = nothing


import Base.==
function ==(a::Tensor, b::Tensor)
     return a===b
end



== (generic function with 176 methods)

In [7]:
Base.size(x::Tensor) = size(x.data)

Base.getindex(x::Tensor, i...) = getindex(x.data, i...)

Base.setindex!(x::Tensor, v, i...) = setindex!(x.data, v, i...)

In [8]:
x = Tensor([1.0 2.0; 3.0 4.0])

println(x)

println(size(x))
# output: (2,2)

x[2,2] = 5.0

println(x)

println(x[2,2,])


Tensor([1.0 2.0; 3.0 4.0])
(2, 2)
Tensor([1.0 2.0; 3.0 5.0])
5.0
